#### LSTM-base-language-model

- Model:
- Data: 

In [1]:
import tensorflow as tf

import numpy as np

import preprocess_helper
import embedding 

In [ ]:
# @TODO define a class Data
class Data():
  
  def __init__(self):
    self.config = Config()
    

In [2]:
# define a class Config 
class Config():
  
  def __init__(self, path_to_config_file=None):
    if path_to_config_file:
      read(self, path_to_config_file)
    else:
      # default constructor 
      self.sentence_length = 30        # number of words per sentence (ie. how many times we should enroll the network)
      self.batch_size = 64             # number of sentences analysed per batch 
      self.embedding_dimensions = 100  # dimension of the embedding 
      self.state_size = 512            # dimension of the hidden state 
      self.max_grad_norm = 10          # max norm of the gradient 
      self.vocabulary_size = 20000     # vocabulary size 
      self.number_of_epochs = 10       # number of epochs used during training 
      self.learning_rate = 1
    
  def read(self, path_to_config_file):
    # implement fc to read config params 
    raise ValueError('Method not implemented yet.')

In [3]:
class Model():
  
  def __init__(self):
    # get all the config params 
    self.config = Config()
    # place holder for a batch dim = sent_len x batch_size (should contain the IDs of each word)
    input_batch = tf.placeholder(tf.int32, [self.config.batch_size, self.config.sentence_length])
    # place holder for the target dim = batch_size x sent_len x 1 (1: id of the target word)
    labels = tf.placeholder(tf.int32, [self.config.batch_size, self.config.sentence_length])
    # var for the embedding of dim = 20k x 100
    embedding = tf.get_variable("embedding", [self.config.vocabulary_size,
                                              self.config.embedding_dimensions],
                                dtype=tf.float32)
    # look-up table st. dim = sent_len x batch_size x emb_size 
    inputs = tf.nn.embedding_lookup(embedding,
                                    input_batch)
    # construct basic LSTM cell 
    lstm_cell = tf.contrib.rnn.BasicLSTMCell(self.config.state_size)
    # build-in tensorflow function to enroll the LSTM
    self.initial_state = lstm_cell.zero_state(self.config.batch_size, tf.float32)
    inputs = tf.unstack(inputs, num=self.config.sentence_length, axis=1)
    output, state = tf.nn.static_rnn(lstm_cell, inputs, dtype=tf.float32)
    output = tf.reshape(output, [self.config.sentence_length*self.config.batch_size, self.config.state_size])
    # project state size on the vocab size dim = state_size x vocabulary_size 
    weights = tf.get_variable("weights", [self.config.state_size,
                                          self.config.vocabulary_size],
                              dtype=tf.float32)
    # add a bias dim = vocabulary_size 
    bias = tf.get_variable("bias", [self.config.vocabulary_size], dtype=tf.float32)
    # compute the logits 
    logits = tf.matmul(output, weights) + bias
    # reshape logits to dim = batch_size x num_steps x vocabulary_size 
    logits = tf.reshape(logits, [self.config.sentence_length,
                                 self.config.batch_size,
                                 self.config.vocabulary_size])
    # define proba with softmax layer with dim = batch_size x num_steps x vocabulary_size 
    probabilities = tf.nn.softmax(logits)
    # should probably reshape before 
    loss = tf.losses.sparse_softmax_cross_entropy(labels,
                                                  logits)
    loss = tf.reduce_sum(loss)
    # optimizer and minimize ...
    optimizer = tf.train.AdamOptimizer(learning_rate=self.config.learning_rate)
    minimize =  optimizer.minimize(loss)

In [7]:

model = Model()

# Display all the trainable variables 
tf.trainable_variables()
# Display all the variables in the session 
for v in tf.get_default_graph().as_graph_def().node:
  print(v.name)

Placeholder
Placeholder_1
embedding/Initializer/random_uniform/shape
embedding/Initializer/random_uniform/min
embedding/Initializer/random_uniform/max
embedding/Initializer/random_uniform/RandomUniform
embedding/Initializer/random_uniform/sub
embedding/Initializer/random_uniform/mul
embedding/Initializer/random_uniform
embedding
embedding/Assign
embedding/read
embedding_lookup
BasicLSTMCellZeroState/Const
BasicLSTMCellZeroState/Const_1
BasicLSTMCellZeroState/concat/axis
BasicLSTMCellZeroState/concat
BasicLSTMCellZeroState/zeros/Const
BasicLSTMCellZeroState/zeros
BasicLSTMCellZeroState/Const_2
BasicLSTMCellZeroState/Const_3
BasicLSTMCellZeroState/Const_4
BasicLSTMCellZeroState/Const_5
BasicLSTMCellZeroState/concat_1/axis
BasicLSTMCellZeroState/concat_1
BasicLSTMCellZeroState/zeros_1/Const
BasicLSTMCellZeroState/zeros_1
BasicLSTMCellZeroState/Const_6
BasicLSTMCellZeroState/Const_7
unstack
rnn/BasicLSTMCellZeroState/Const
rnn/BasicLSTMCellZeroState/Const_1
rnn/BasicLSTMCellZeroState/conca

rnn/rnn/basic_lstm_cell/MatMul_28
rnn/rnn/basic_lstm_cell/BiasAdd_28
rnn/rnn/basic_lstm_cell/Const_28
rnn/rnn/basic_lstm_cell/split_28/split_dim
rnn/rnn/basic_lstm_cell/split_28
rnn/rnn/basic_lstm_cell/add_56/y
rnn/rnn/basic_lstm_cell/add_56
rnn/rnn/basic_lstm_cell/Sigmoid_84
rnn/rnn/basic_lstm_cell/mul_84
rnn/rnn/basic_lstm_cell/Sigmoid_85
rnn/rnn/basic_lstm_cell/Tanh_56
rnn/rnn/basic_lstm_cell/mul_85
rnn/rnn/basic_lstm_cell/add_57
rnn/rnn/basic_lstm_cell/Tanh_57
rnn/rnn/basic_lstm_cell/Sigmoid_86
rnn/rnn/basic_lstm_cell/mul_86
rnn/rnn/basic_lstm_cell/concat_29/axis
rnn/rnn/basic_lstm_cell/concat_29
rnn/rnn/basic_lstm_cell/MatMul_29
rnn/rnn/basic_lstm_cell/BiasAdd_29
rnn/rnn/basic_lstm_cell/Const_29
rnn/rnn/basic_lstm_cell/split_29/split_dim
rnn/rnn/basic_lstm_cell/split_29
rnn/rnn/basic_lstm_cell/add_58/y
rnn/rnn/basic_lstm_cell/add_58
rnn/rnn/basic_lstm_cell/Sigmoid_87
rnn/rnn/basic_lstm_cell/mul_87
rnn/rnn/basic_lstm_cell/Sigmoid_88
rnn/rnn/basic_lstm_cell/Tanh_58
rnn/rnn/basic_lst

gradients/rnn/rnn/basic_lstm_cell/mul_43_grad/Shape_1
gradients/rnn/rnn/basic_lstm_cell/mul_43_grad/BroadcastGradientArgs
gradients/rnn/rnn/basic_lstm_cell/mul_43_grad/mul
gradients/rnn/rnn/basic_lstm_cell/mul_43_grad/Sum
gradients/rnn/rnn/basic_lstm_cell/mul_43_grad/Reshape
gradients/rnn/rnn/basic_lstm_cell/mul_43_grad/mul_1
gradients/rnn/rnn/basic_lstm_cell/mul_43_grad/Sum_1
gradients/rnn/rnn/basic_lstm_cell/mul_43_grad/Reshape_1
gradients/rnn/rnn/basic_lstm_cell/mul_43_grad/tuple/group_deps
gradients/rnn/rnn/basic_lstm_cell/mul_43_grad/tuple/control_dependency
gradients/rnn/rnn/basic_lstm_cell/mul_43_grad/tuple/control_dependency_1
gradients/rnn/rnn/basic_lstm_cell/Sigmoid_42_grad/SigmoidGrad
gradients/rnn/rnn/basic_lstm_cell/Sigmoid_43_grad/SigmoidGrad
gradients/rnn/rnn/basic_lstm_cell/Tanh_28_grad/TanhGrad
gradients/rnn/rnn/basic_lstm_cell/add_28_grad/Shape
gradients/rnn/rnn/basic_lstm_cell/add_28_grad/Shape_1
gradients/rnn/rnn/basic_lstm_cell/add_28_grad/BroadcastGradientArgs
gra

In [ ]:
def build_lstm_graph(lstm_cell, input_batch):
  # cell: tensorflow LSTM object
  # batch: tensorflow of shape sent_len x batch_size x emb_dim
  
  # return value is a N-D tensor of shape [batch_size, state_size] filled with zeros.
  initial_state = lstm_cell.zero_state(batch_size, tf.float32)
  
  # init state is the init one 
  state = initial_state

  # where to store the cell_output after each time_step 
  outputs = []
  
  for time_step in range(num_steps):
    # see if there is a better way to reuse the variables 
    if time_step > 0:
      tf.get_variable_scope().reuse_variables()
    # given the current state 'state' and the input_batch, compute the new state and the cell_output 
    #  - cell_output: (batch_size, output_size) What is output_size here ??
    #  - state:  (batch_size, state_size)
    cell_output, state = lstm_cell(input_batch[:, time_step, :], state)
    outputs.append(cell_output)

  # tf.reshape(tensor, shape)
  # tf.concat(values, axis)
  output = tf.reshape(tf.concat(outputs, 1), [-1, state_size])
  
  return output, state

def generate_emdedding():
  embedding = np.random.uniform(low=0, high=1, size=embedding_dimensions)
  return embedding 

In [ ]:
# NOW NICELY REORGANIZSED INTO THE CLASS MODEL...
# DOES NOT COMPILE 
sentence_length = 30        # number of words per sentence (ie. how many times we should enroll the network)
batch_size = 64             # number of sentences analysed per batch 
embedding_dimensions = 100  # dimension of the embedding 
state_size = 512            # dimension of the hidden state 
max_grad_norm = 10          # max norm of the gradient 
vocabulary_size = 20000     # vocabulary size 

embedding = np.random.uniform(low=0, high=1, size=embedding_dimensions)

# place holder for a batch dim = sent_len x batch_size x emb_dim
input_batch = tf.placeholder(tf.float32, [sentence_length, batch_size, embedding_dimensions])

# place holder for the target dim = sent_len x batch_size x 1 (1: id of the target word)
labels = tf.placeholder(tf.float32, [sentence_length, batch_size, 1])

# var for the embedding of dim=20k x 100
embedding = tf.get_variable("embedding", [vocabulary_size, embedding_dimensions], dtype=tf.float32)

# this i don't know what it is
inputs = tf.nn.embedding_lookup(embedding,
                                input_data)

# construct basic LSTM cell 
lstm_cell = tf.contrib.rnn.BasicLSTMCell(lstm_size)

# build-in tensorflow function  
output, state = tf.nn.static_rnn(lstm_celllstm_celllstm_cell, batch, dtype=tf.float32)
# our own implementation of tf.nn.static_rnn
#output, state = build_lstm_graph(lstm_celllstm_cell, batch)

# project state size on the vocab size dim = state_size x vocabulary_size 
weights = tf.get_variable("weights", [state_size, vocabulary_size], dtype=tf.float32)

# add a bias dim = vocabulary_size 
bias = tf.get_variable("bias", [vocabulary_size], dtype=data_type())

# compute the logits 
logits = tf.matmul(output, weights) + bias

# reshape logits to dim = batch_size x num_steps x vocabulary_size 
logits = tf.reshape(logits, [batch_size, num_steps, vocabulary_size])

# define proba with softmax layer with dim = batch_size x num_steps x vocabulary_size 
probabilities = tf.nn.softmax(logits)

# should probably reshape before 
loss = tf.losses.sparse_softmax_cross_entropy(
            labels,
            logits,
            weights=1.0,
            scope=None,
            loss_collection=tf.GraphKeys.LOSSES,
            reduction=Reduction.SUM_BY_NONZERO_WEIGHTS)

loss = tf.reduce_sum(loss)

# Calling minimize() takes care of both computing the gradients and applying them to the variables.
# If you want to process the gradients before applying them you can instead use the optimizer in four steps:
#   1- Define an optimizer 
#   2- Compute the gradients with compute_gradients().
#   3- Process the gradients as you wish.
#   4- Apply the processed gradients with apply_gradients()

# So instead of:
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
minimize =  optimizer.minimize(loss)

# Use:
# 1) define the optimizer
#optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
# 2) compute the gradients
#gradients_and_vars = optimizer.compute_gradients(loss)
# 3) process the gradient 
#processed_gradients_and_vars = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients_and_vars]
# 4) apply the gradients.
#train_optimizer = optimizer.apply_gradients(processed_gradients_and_vars)


In [ ]:
# simplified version of the Tensorflow tutorial on RNNs
# available at: https://www.tensorflow.org/tutorials/recurrent

def main():
  
  max_epoch = 1

  frequent_words = load_frequent_words('google-10000-english/20k.txt')
  test_data = load_and_process_data('data/sentences.eval', frequent_words, 28)
  train_data = load_and_process_data('data/sentences.train', frequent_words, 28)

  config = get_config()
  eval_config = get_config()
  eval_config.batch_size = 1
  eval_config.num_steps = 1

  with tf.Graph().as_default():
    
    # init with Xavier
    initializer = tf.contrib.layers.xavier_initializer()

    with tf.name_scope("Train"):
      train_input = train_data # @TODO add the labels here 
      with tf.variable_scope("Model", reuse=None, initializer=initializer):
        training_model = Model(is_training=True,
                               config=config,
                               input_=train_input)
      tf.summary.scalar("Training Loss", training_model.loss)
      tf.summary.scalar("Learning Rate", training_model.learning_rate)

    with tf.name_scope("Test"):
      test_input = test_data # @TODO add the labels here 
      with tf.variable_scope("Model", reuse=True, initializer=initializer):
        testing_model = Model(is_training=False,
                              config=eval_config,
                              input_=test_input)

    for i in range(max_epoch):
      print("Epoch: %d Learning rate: %.3f" % (i + 1, session.run(m.lr)))
      train_perplexity = run_epoch(session, training_model,
                                   eval_op=training_model.train_op,
                                   verbose=True)
      print("Epoch: %d Train Perplexity: %.3f" % (i + 1, train_perplexity))

      test_perplexity = run_epoch(session, mtest)
      print("Test Perplexity: %.3f" % test_perplexity)


if __name__ == "__main__":
  tf.app.run()


In [ ]:
batch_size = 64
number_of_batches = int(len(train_input)/batch_size) # = 31250
epoch = 10

init_op = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init_op)

for i in range(epoch):
    ptr = 0
    for j in range(no_of_batches):
        inp, out = train_input[ptr:ptr+batch_size], train_output[ptr:ptr+batch_size]
        ptr+=batch_size
        sess.run(minimize,{data: inp, target: out})
    print("Epoch - ",str(i))
incorrect = sess.run(error,{data: test_input, target: test_output})
print('Epoch {:2d} error {:3.1f}%'.format(i + 1, 100 * incorrect))
sess.close()